In [1]:
# import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")
 
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir_ori = Path(r'Y:\coskun-lab\Thomas\15_PLA\data')
data_dir = data_dir = (Path().cwd().parents[0] / 'data').absolute()


df_meta_path = data_dir_ori /'OCT Mouse' / 'Whole' / 'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_meta_path)


In [5]:
df_imgs

,Timepoint,FOV,Path
0,1M,FW1,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\Who...
1,1M,FW2,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\Who...
2,1W,FW1,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\Who...
3,1W,FW2,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\Who...
4,1W,FW3,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\Who...


# Read PPI data

In [6]:
# Read PPI 
PPI_save_path =  data_dir_ori / 'OCT Mouse' / 'Whole' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)
df = df.rename(columns={'x': 'row', 'y': 'column'})
df.head()

,Cyto,row,column,PPI,Nuclei,Condition,FOV
0,14586,5545,866,TEAD1 & YAP1,14586,1M,FW1
1,14586,5545,866,TEAD1 & YAP1,14586,1M,FW1
2,14586,5545,866,TEAD1 & YAP1,14586,1M,FW1
3,14586,5545,866,TEAD1 & YAP1,14586,1M,FW1
4,1252,521,13171,TEAD1 & YAP1,0,1M,FW1


In [7]:
df.PPI.unique()

array(['TEAD1 & YAP1', 'CylinE & CDK2', 'P-ERK & c-MYC', 'p-AKT & mTOR',
       'Mcl-1 & BAK'], dtype=object)

# Plot FOV scatter 

In [8]:
# Helper functions
import napari
import vispy.color
import networkx as nx
from copy import deepcopy 

def read_pkl(path):
    with open(path, 'rb') as f:
        item = pickle.load(f)
    return item

def plot_network(g, label, ax, **kwargs):
    pos = nx.get_node_attributes(g, 'pos')
    pos_T = {k: v[[1,0]] for k,v in pos.items()}
    
    nx.draw(g, pos_T, node_color=label, with_labels=False, ax=ax, **kwargs)
    return ax
    
def get_loc_ppi(df, Condition, FOV, PPI_name):
    df_ppi = df[(df.Condition == Condition) & (df.FOV == FOV)]
    df_ppi = df_ppi[df_ppi.PPI == PPI_name]
    points = df_ppi[['row', 'column']].to_numpy()
    return points

# remove first and last consecutive zeros along dimension
def removeFirstAndLast(zeroArray):
    
    # remove first
    for ii, val in enumerate(zeroArray):
        if np.sum(zeroArray[:ii + 1]) != ii + 1: # all consecutive trues
            break # exit loop
            
        else:
            lowest = ii # index of last of the first consecutive zeros
            
    # remove last
    for ii, val in enumerate(np.flip(zeroArray)):
        if np.sum(zeroArray[:ii + 1]) != ii + 1: # all consecutive trues
            break # exit loop
            
        else:
            highest = len(zeroArray) - ii # index of first of the last consecutive zeros            
    
    return (lowest, highest)

# remove black borders from napari screenshot (2D image)
def removeBorders(RGB):

    rowMin = []
    rowMax = []
    colMin = []
    colMax = []
    for jj in range(3): # each RGB
        
        img = RGB[:, :, jj]
        # find first and last rows and cols of all zeros
        # remove any rows of zeros
        zeroRows = np.all(img == 0, axis = 1)
        zeroCols = np.all(img == 0, axis = 0)

        posRows = removeFirstAndLast(zeroRows)
        posCols = removeFirstAndLast(zeroCols)            
        
        rowMin.append(posRows[0])
        rowMax.append(posRows[1])
        colMin.append(posCols[0])
        colMax.append(posCols[1])
        
    # make sure border signal is not cut out
    rowMin = np.min(rowMin)
    rowMax = np.max(rowMax)
    colMin = np.min(colMin)
    colMax = np.max(colMax)
        
    cropped = RGB[rowMin: rowMax, colMin: colMax, :] 
    
    return cropped

def plot_PPI_scatter(points, colors, c_limit_dict, img, contour=None, contour_nuclei=None):
    # Napari viewer
    viewer = napari.view_image(img[0], colormap='blue', contrast_limits=[0, 0.7])
    for j, point in enumerate(points):
        points_layer = viewer.add_points(point, size=10,face_color=colors[j])
    
    if contour is not None:
        shapes_layer = viewer.add_shapes(contour, shape_type='polygon', edge_width=1,
                            edge_color='red', face_color=[0,0,0,0])
    if contour_nuclei is not None:
        shapes_layer = viewer.add_shapes(contour_nuclei, shape_type='polygon', edge_width=1,
                            edge_color='blue', face_color=[0,0,0,0])
    # Get screenshot
    screenshot = viewer.screenshot(size=np.array((img.shape[1], img.shape[2])))
    screenshot1 = removeBorders(screenshot)
    viewer.close()
    return screenshot1

In [9]:
image_dir = (Path().cwd().parents[0] / 'figures').absolute() / 'tissue_xeno_scatter'
image_dir.mkdir(parents=True, exist_ok=True)

In [10]:
PPIs = ['TEAD1 & YAP1', 'CylinE & CDK2', 'P-ERK & c-MYC', 'p-AKT & mTOR',
       'Mcl-1 & BAK']

In [11]:
import skimage.io

In [13]:
from matplotlib.colors import LinearSegmentedColormap
group = df_imgs.groupby(['Timepoint', 'FOV'])

# Loop through Condition FOV
for name, df_group in group:
    # Get imgs shape >
    path = df_imgs[(df_imgs.Timepoint == name[0]) & (df_imgs.FOV == name[1])].Path.item()
    with h5py.File(path, "r") as f:
        imgs = f['imgs'][:]
        markers = f['imgs'].attrs['Marker']

    # Get contrast limits range
    c_limit_dict = {}
    for n in np.unique(markers):
        indice = list(markers).index(n)
        max_c = np.quantile(imgs[indice], 1.0)
        c_limit_dict[n] = [0, max_c]

    # Plot PPI
    for PPI in PPIs:
        points = get_loc_ppi(df, name[0], name[1], PPI)
        screenshot = plot_PPI_scatter([points], ['magenta'], c_limit_dict, imgs)
        skimage.io.imsave(image_dir / f'{name[0]}_{name[1]}_{PPI}.png', screenshot)
